## Setup configuration file and parameters
Prepare or chose a config file and set the parameters to your needs

In [1]:
import json
from src.FunctionalValidation import FunctionalValidation
from src.ControlLogicGenerator import ControlLogicGenerator
from src.Simulator import Simulator
from src.Plotter import Plotter
from src.util import copy_pwm_files

############## PARAMETERS ###################
max_dev = 50    # Default 50
dev_wf = 30     # Default 20
fig_type = ['pdf', 'png', 'svg'][1]
config_file = './configs/SSAx1.json'

with open(config_file, 'r') as f:
    config = json.load(f)

C:\Users\Shype\Desktop\Paper in work\[In Work] MASAM\src\Plotter.py:169: SyntaxWarning: invalid escape sequence '\m'
  axs[idx].set_xlabel("Time in $\mu$s", fontsize=24, labelpad=0)


Verify the functionality of the algorithm and present a state history of each memristor:

In [2]:
# Check if the algorithm is valid and the resulting states are correct
Verifier = FunctionalValidation(config) 
Verifier.calc_algorithm(plot_tt=True)
print("\n--------------- Logic States verified! ---------------\n")

2024-08-09 16:18:55,372 - Simulation - INFO - Initializing class LogicState
2024-08-09 16:18:55,378 - Simulation - INFO - Applied Operations: ['F[w1]']
2024-08-09 16:18:55,379 - Simulation - INFO - Applied Operations: ['F[w1]', 'I[b,w1]']
2024-08-09 16:18:55,380 - Simulation - INFO - Applied Operations: ['F[w1]', 'I[b,w1]', 'I[a,w1]']
2024-08-09 16:18:55,380 - Simulation - INFO - Applied Operations: ['F[w1]', 'I[b,w1]', 'I[a,w1]', 'F[a]', 'I[w1,c]']
2024-08-09 16:18:55,381 - Simulation - INFO - Applied Operations: ['F[w1]', 'I[b,w1]', 'I[a,w1]', 'F[a]', 'I[w1,c]', 'I[c,a]', 'F[w1]']


#####################################
Applied operations: ['F[w1]']
a | b | c | w1|
--------------------
0 | 0 | 0 | 0
0 | 0 | 1 | 0
0 | 1 | 0 | 0
0 | 1 | 1 | 0
1 | 0 | 0 | 0
1 | 0 | 1 | 0
1 | 1 | 0 | 0
1 | 1 | 1 | 0
#####################################
#####################################
Applied operations: ['F[w1]', 'I[b,w1]']
a | b | c | w1|
--------------------
0 | 0 | 0 | 1
0 | 0 | 1 | 1
0 | 1 | 0 | 0
0 | 1 | 1 | 0
1 | 0 | 0 | 1
1 | 0 | 1 | 1
1 | 1 | 0 | 0
1 | 1 | 1 | 0
#####################################
#####################################
Applied operations: ['F[w1]', 'I[b,w1]', 'I[a,w1]']
a | b | c | w1|
--------------------
0 | 0 | 0 | 1
0 | 0 | 1 | 1
0 | 1 | 0 | 1
0 | 1 | 1 | 1
1 | 0 | 0 | 1
1 | 0 | 1 | 1
1 | 1 | 0 | 0
1 | 1 | 1 | 0
#####################################
#####################################
Applied operations: ['F[w1]', 'I[b,w1]', 'I[a,w1]', 'F[a]', 'I[w1,c]']
a | b | c | w1|
--------------------
0 | 0 | 0 | 1
0 | 0 | 1 | 1
0 | 1 | 0 | 1
0 | 1 | 1 | 1


## Automatically create PWM signals
Instantiate a PWMWriter class, automatically write PWM files for the chosen algorithm in the defined topology, and temporary store them in the **PWM_output** 
folder for debugging:


In [3]:
# Automatically create PWM signals and store them in "PWM_output"
CLG = ControlLogicGenerator(config)
CLG.eval_algo()
print("\n--------------- PWM Signals created! ---------------\n")

2024-08-09 16:19:36,140 - Simulation - INFO - Initializing PWM Writer
2024-08-09 16:19:36,145 - Simulation - INFO - File ./outputs/PWM_output/a.csv created!
2024-08-09 16:19:36,145 - Simulation - INFO - File ./outputs/PWM_output/b.csv created!
2024-08-09 16:19:36,146 - Simulation - INFO - File ./outputs/PWM_output/c.csv created!
2024-08-09 16:19:36,146 - Simulation - INFO - File ./outputs/PWM_output/w1.csv created!
2024-08-09 16:19:36,147 - Simulation - INFO - File ./outputs/PWM_output/a_sw.csv created!
2024-08-09 16:19:36,147 - Simulation - INFO - File ./outputs/PWM_output/b_sw.csv created!
2024-08-09 16:19:36,148 - Simulation - INFO - File ./outputs/PWM_output/c_sw1.csv created!
2024-08-09 16:19:36,148 - Simulation - INFO - File ./outputs/PWM_output/c_sw2.csv created!
2024-08-09 16:19:36,149 - Simulation - INFO - File ./outputs/PWM_output/w1_sw1.csv created!
2024-08-09 16:19:36,149 - Simulation - INFO - File ./outputs/PWM_output/w1_sw2.csv created!
2024-08-09 16:19:36,154 - Simulatio

File ./outputs/PWM_output/a.csv created!
File ./outputs/PWM_output/b.csv created!
File ./outputs/PWM_output/c.csv created!
File ./outputs/PWM_output/w1.csv created!
File ./outputs/PWM_output/a_sw.csv created!
File ./outputs/PWM_output/b_sw.csv created!
File ./outputs/PWM_output/c_sw1.csv created!
File ./outputs/PWM_output/c_sw2.csv created!
File ./outputs/PWM_output/w1_sw1.csv created!
File ./outputs/PWM_output/w1_sw2.csv created!
PWM Files written successfully

--------------- PWM Signals created! ---------------



Overwrite the current algorithm in the chosen topology folder:

In [4]:
# Copy the files to the folder of the corresponding topology (This removes the old files !)
copy_pwm_files(config, CLG.step_size)
print(f"\n--------- Files of {config["topology"]} topology overwritten! --------\n")


--------- Files of Semi-Serial topology overwritten! --------



## Illustrate the deviation experiments
Instantiate a Plotter class in which the Simulator is embedded for convenience. \
The energy consumption is automatically calculated.

In [5]:
PLT = Plotter(config)

2024-08-09 16:19:53,658 - Simulation - INFO - Initializing Plotter
2024-08-09 16:19:53,658 - Simulation - INFO - Initializing Simulator
2024-08-09 16:19:53,674 - Simulation - INFO - Started calculating energy consumption


Calculating energy consumption:


100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
2024-08-09 16:20:01,860 - Simulation - INFO - Average Energy consumption: 1.7030169375e-09
2024-08-09 16:20:01,860 - Simulation - INFO - Energy over Combinations: [9.67572e-10, 1.78572e-09, 1.2769785e-09, 2.0727900000000002e-09, 1.32051e-09, 2.116155e-09, 1.648965e-09, 2.435445e-09]


Average Energy consumption: 1.7030169375e-09
Energy over Combination: [9.67572e-10, 1.78572e-09, 1.2769785e-09, 2.0727900000000002e-09, 1.32051e-09, 2.116155e-09, 1.648965e-09, 2.435445e-09]


The three main plot options are:
1) plot_deviation_scatter()
2) plot_deviation_range()
3) plot_waveforms_with_deviation()

If the simulations have been run already (The files of the last simulation is stored in the folder **outputs**), the parameter **recompute** can be set **False**.

In [ ]:
PLT.plot_deviation_scatter(max_dev=max_dev, recompute=True, fig_type=fig_type)
PLT.plot_deviation_range(max_dev=max_dev, recompute=False, fig_type=fig_type)
print(f"\n--------- Deviation Experiments completed --------\n")

Create waveform images with deviation ranges for all input combinations with deviation **dev_wf**:

In [ ]:
for comb in range(8):
    comb_str = bin(comb)[2:].zfill(3)
    PLT.plot_waveforms_with_deviation(comb_str, dev=dev_wf, recompute=False, fig_type=fig_type)
print(f"\n--------- Waveforms with deviation {dev_wf} saved --------\n")

Illustrate a single input combination with unique parameters:

In [ ]:
comb = '100'
dev = 20
fig_type_s = 'png'
recompute_s = True
PLT.plot_waveforms_with_deviation(comb, dev=dev, recompute=recompute_s, fig_type=fig_type_s, plots_per_subfigure=4)

## Store the results
Save created images and state deviation files in **OUTPUT_FILES**


In [5]:
PLT.save_algorithm_files(f"{config["algorithm"].split(".")[0]}")